In [2]:
!wget --output-document=ml-100k.zip http://www.grouplens.org/system/files/ml-100k.zip
!unzip ml-100k.zip

--2020-10-17 15:53:22--  http://www.grouplens.org/system/files/ml-100k.zip
Resolving www.grouplens.org (www.grouplens.org)... 128.101.34.235
Connecting to www.grouplens.org (www.grouplens.org)|128.101.34.235|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://grouplens.org/system/files/ml-100k.zip [following]
--2020-10-17 15:53:22--  https://grouplens.org/system/files/ml-100k.zip
Resolving grouplens.org (grouplens.org)... 128.101.34.235
Connecting to grouplens.org (grouplens.org)|128.101.34.235|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://files.grouplens.org/papers/ml-100k.zip [following]
--2020-10-17 15:53:23--  http://files.grouplens.org/papers/ml-100k.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4924029 (4.7M) [appl

This data set consists of:  
	* 100,000 ratings (1-5) from 943 users on 1682 movies.   
	* Each user has rated at least 20 movies.   

In [1]:
import pandas as pd

direc = '/content/ml-100k/'

headers = "movie id | movie title | release date | video release date | IMDb URL | unknown | Action | Adventure | Animation | Children's | Comedy | Crime | Documentary | Drama | Fantasy | Film-Noir | Horror | Musical | Mystery | Romance | Sci-Fi | Thriller | War | Western | "
movies = pd.read_table(direc + 'u.item',sep = '|',names = headers.split(" | "), engine = 'python')
display(movies.tail())

ratings_raw = pd.read_table(direc + 'ua.base',sep = '\t', names = ['user_id','movie_id','rating','timestamp'], engine = 'python')
display(ratings_raw)

,movie id,movie title,release date,video release date,IMDb URL,unknown,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,
1677,1678,Mat' i syn (1997),06-Feb-1998,NaN,http://us.imdb.com/M/title-exact?Mat%27+i+syn+...,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,NaN
1678,1679,B. Monkey (1998),06-Feb-1998,NaN,http://us.imdb.com/M/title-exact?B%2E+Monkey+(...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,NaN
1679,1680,Sliding Doors (1998),01-Jan-1998,NaN,http://us.imdb.com/Title?Sliding+Doors+(1998),0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,NaN
1680,1681,You So Crazy (1994),01-Jan-1994,NaN,http://us.imdb.com/M/title-exact?You%20So%20Cr...,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN
1681,1682,Scream of Stone (Schrei aus Stein) (1991),08-Mar-1996,NaN,http://us.imdb.com/M/title-exact?Schrei%20aus%...,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,NaN


,user_id,movie_id,rating,timestamp
0,1,1,5,874965758
1,1,2,3,876893171
2,1,3,4,878542960
3,1,4,3,876893119
4,1,5,3,889751712
...,...,...,...,...
90565,943,1047,2,875502146
90566,943,1074,4,888640250
90567,943,1188,3,888640250
90568,943,1228,3,888640275


In [2]:
# movies.query('unknown == 1') # 장르가 알려지지 않은 경우는 제외하기로 하자.

In [3]:
# movies = movies.query('unknown != 1')
movies.drop(["release date","video release date","IMDb URL",'unknown'], axis = 1, inplace = True)

In [4]:
# genres = movies.iloc[:,2:].apply(lambda x: [g for g,v in x.items() if v == 1], axis = 1) # 장르는 뭉쳐서 넣자
genres = movies.iloc[:,2:].apply(lambda x:"|".join([g for g,v in x.items() if v == 1]), axis = 1)

In [5]:
movies = pd.concat([movies.iloc[:,:2], genres], axis = 1)
movies.rename(columns = {0:'genres'}, inplace=True)
movies['genres'] = movies.genres.str.split('|')
movies

,movie id,movie title,genres
0,1,Toy Story (1995),"[Animation, Children's, Comedy]"
1,2,GoldenEye (1995),"[Action, Adventure, Thriller]"
2,3,Four Rooms (1995),[Thriller]
3,4,Get Shorty (1995),"[Action, Comedy, Drama]"
4,5,Copycat (1995),"[Crime, Drama, Thriller]"
...,...,...,...
1677,1678,Mat' i syn (1997),[Drama]
1678,1679,B. Monkey (1998),"[Romance, Thriller]"
1679,1680,Sliding Doors (1998),"[Drama, Romance]"
1680,1681,You So Crazy (1994),[Comedy]


In [6]:
ratings = ratings_raw.copy()
ratings.drop(['timestamp', 'rating'], axis = 1, inplace = True)
ratings

,user_id,movie_id
0,1,1
1,1,2
2,1,3
3,1,4
4,1,5
...,...,...
90565,943,1047
90566,943,1074
90567,943,1188
90568,943,1228


# find_path
- Entity : movie, user, genre
- Entity types : user, movie, type
- Relations : rate, rate_, category, category_
  > User-`rate`-Movie, Movie-`rate_`-User  
  > Genres-`category_`-Movie, Movie-`category`-Genres



In [8]:
ratings['u_user_id'] = ratings.user_id.map(lambda x: f'u{x}')
ratings['m_movie_id'] = ratings.movie_id.map(lambda x: f'm{x}')
ratings

,user_id,movie_id,u_user_id,m_movie_id
0,1,1,u1,m1
1,1,2,u1,m2
2,1,3,u1,m3
3,1,4,u1,m4
4,1,5,u1,m5
...,...,...,...,...
90565,943,1047,u943,m1047
90566,943,1074,u943,m1074
90567,943,1188,u943,m1188
90568,943,1228,u943,m1228


In [10]:
user_to_movies = ratings.groupby('u_user_id').m_movie_id.apply(list).to_dict()
user_to_movies['u1'][:5]

['m1', 'm2', 'm3', 'm4', 'm5']

In [11]:
movies['m_movie_id'] = movies['movie id'].map(lambda x: f'm{x}')
movies

,movie id,movie title,genres,m_movie_id
0,1,Toy Story (1995),"[Animation, Children's, Comedy]",m1
1,2,GoldenEye (1995),"[Action, Adventure, Thriller]",m2
2,3,Four Rooms (1995),[Thriller],m3
3,4,Get Shorty (1995),"[Action, Comedy, Drama]",m4
4,5,Copycat (1995),"[Crime, Drama, Thriller]",m5
...,...,...,...,...
1677,1678,Mat' i syn (1997),[Drama],m1678
1678,1679,B. Monkey (1998),"[Romance, Thriller]",m1679
1679,1680,Sliding Doors (1998),"[Drama, Romance]",m1680
1680,1681,You So Crazy (1994),[Comedy],m1681


In [15]:
movie_to_user_genre = movies.set_index('m_movie_id').genres.to_dict()
for uid, movie_list in user_to_movies.items():
    for movie in movie_list:
        movie_to_user_genre[movie].append(uid)

movie_to_user_genre['m1'][:5]

['Animation', "Children's", 'Comedy', 'u1', 'u10']

In [18]:
from collections import defaultdict

movie_to_genre = movies.set_index('m_movie_id').genres.to_dict()
genre_to_movies = defaultdict(list)
for movie, genres in movie_to_genre.items():
    for genre in genres:
        genre_to_movies[genre].append(movie)

genre_to_movies['Animation'][:5]

['m1', 'm71', 'm95', 'm99', 'm101']

In [48]:
positive_path = []

def get_relation(head_entity, end_entity):
    # 나중에 test 할 때 새로들어오는 movie/genre/user id에 대한 처리도 넣어주자
    # print(head_entity, end_entity)
    head_entity, end_entity = head_entity[0], end_entity[0]

    if head_entity == 'm':
        if end_entity == 'u':
            # return relation_dict["_rate"]
            return 'rate'
        else: # genre
            return 'category'

    elif head_entity == 'u':
        # if end_entity == 'm': # 무조건 movie일테니까
        return 'rate_'

    else: # genre
        return 'category_'

def find_path(head_entity, next_entities, depth): # init ; user_behavior

    if depth >= 3:
        for entity in next_entities:
            if visited[entity]:
                continue
            if entity[0] == 'm' and entity in user_behavior:
                positive_path.append(path + [entity])
                # print(" ".join(map(str,path + [entity])))
            # else:
            #     negative_path.append(path + [movie])
        return
    
    path.append(head_entity)
    for entity in next_entities: # movie에서 갈 수 있는 모든 entity. user들, genre들
        if visited[entity]:
            continue
        visited[entity] = True
        path.append(get_relation(head_entity, entity))

        if entity[0] == 'u':
            # 이 user가 rate한 item list로 이동한다.
            find_path(entity, user_to_movies[entity], depth+1)
        elif entity[0] == 'm':
            find_path(entity, movie_to_user_genre[entity], depth + 1)
        else: # i.e. type is genre:
            find_path(entity, genre_to_movies[entity], depth + 1)

        path.pop() # relation 제거

    path.pop() # head_entity 제거

In [49]:
import time

path = []
visited = defaultdict(lambda : False)

start = time.time()

user = 'u8'
user_behavior = user_to_movies[user]

find_path(user, user_behavior, 1)

print(time.time() - start)

0.08255434036254883


In [50]:
print(positive_path[0])
print(positive_path[-1])

['u8', 'rate_', 'm7', 'category', 'm55']
['u8', 'rate_', 'm510', 'rate', 'm566']


In [51]:
len(positive_path) # 2만개의 path를 찾는데에 1초 이하가 소요된다.

8568

In [52]:
paths = pd.DataFrame(positive_path, columns = ['movie_id_1','genre_1','movie_id_2','genre_2','movie_id_3'])
paths

,movie_id_1,genre_1,movie_id_2,genre_2,movie_id_3
0,u8,rate_,m7,category,m55
1,u8,rate_,m7,category,m56
2,u8,rate_,m7,category,m127
3,u8,rate_,m7,category,m172
4,u8,rate_,m7,category,m187
...,...,...,...,...,...
8563,u8,rate_,m358,rate,m689
8564,u8,rate_,m358,rate,m689
8565,u8,rate_,m358,rate,m688
8566,u8,rate_,m358,rate,m689


In [53]:
# 같은 user-item 사이의 path는 보통 몇개씩있을까?

# 보통 약 161개의 path가 발견된다.

paths.groupby('movie_id_3').apply(lambda x: len(x)).describe()

count     48.000000
mean     178.500000
std       92.644367
min        9.000000
25%      116.750000
50%      161.500000
75%      234.000000
max      414.000000
dtype: float64